<center><h1>Xie_Qinghui_HW3</h1></center>
<br>
<br>

Name: Qinghui Xie
<br>
Github Username: qxie44
<br>
USC ID: 7956658480

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats

Get the AReM Data Set

In [2]:
pathToData = '../data/AReM'
#Bending1 - 7ds, Bending2 - 7ds, others- 15ds
activity = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']


### (b) Test and Train Data

In [25]:
df4 = pd.read_csv('../data/AReM/bending2/dataset4.csv', delim_whitespace=True, skiprows=5, skipfooter=1, engine='python', header=None)
df4.columns = ['# Columns: time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
df4.head(10)
#data.to_csv('cleaned_bending2_data.csv', index=False)
#chatgpt: how do I replace this newly cleaned file into the existing dataset4 file when going through the loop below:
#"if i == 'bending2':
#                # Use the cleaned df4 DataFrame instead of reading from the file
#                testData.append(df4)
#            else:
#                # Load test data and add a label
#                df = pd.read_csv(file_path, skiprows=4)
#                df['label'] = i  # Use the activity name as the label
#               testData.append(df)"

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,32.50,0.50,0.00,0.00,19.00,1.00
1,250,32.50,0.50,0.00,0.00,18.50,0.50
2,500,32.75,0.43,1.00,0.00,18.00,0.00
3,750,32.50,0.50,0.00,0.00,17.50,0.50
4,1000,32.50,0.50,7.50,0.50,17.50,0.87
5,1250,32.67,0.47,11.00,1.00,16.75,0.83
6,1500,32.50,0.50,6.25,0.83,18.00,0.00
7,1750,32.50,0.50,3.50,0.87,18.00,0.00
8,2000,32.33,0.47,6.00,0.00,18.33,0.47
9,2250,32.67,0.47,8.00,0.82,18.00,0.00


In [26]:
#test: for bending keep 1-2, the rest 1-3
#train: the rest
filesForTesting = {
    'bending1': ['dataset1.csv', 'dataset2.csv'],
    'bending2': ['dataset1.csv', 'dataset2.csv'],
    'cycling': ['dataset1.csv', 'dataset2.csv', 'dataset3.csv'],
    'lying': ['dataset1.csv', 'dataset2.csv', 'dataset3.csv'],
    'sitting': ['dataset1.csv', 'dataset2.csv', 'dataset3.csv'],
    'standing': ['dataset1.csv', 'dataset2.csv', 'dataset3.csv'],
    'walking': ['dataset1.csv', 'dataset2.csv', 'dataset3.csv']
}
testData = []
trainData =[]

#loops through all the activites and loads them to testing data
for i in activity:
    path = os.path.join(pathToData, i)
    if i in ['bending1', 'bending2']:
        files_for_test = filesForTesting[i]
    else:
        files_for_test = filesForTesting[i]

    for file_name in os.listdir(path):
        file_path = os.path.join(path, file_name)
        if file_name in files_for_test:
            # Load test data and add a label
            df = pd.read_csv(file_path, skiprows=4)
            testData.append(df)
        else:
            #this specific dataset has wrong delim, replace when it gets to ds4 in bending
            if i == 'bending2' and file_name == 'dataset4.csv':
                trainData.append(df4)
                
            # Load training data and add a label
            else:
                df = pd.read_csv(file_path, skiprows=4, skipfooter=1, engine='python')
                trainData.append(df)
            

In [28]:
for df in trainData[:10]:
    print(df.head(3)) 



   # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                0      42.00       0.00      18.50       0.50      12.00   
1              250      42.00       0.00      18.00       0.00      11.33   
2              500      42.75       0.43      16.75       1.79      18.25   

   var_rss23  
0       0.00  
1       0.94  
2       0.43  
   # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                0      41.25       1.30      24.00       0.00      32.33   
1              250      41.00       1.00      22.50       1.12      33.50   
2              500      44.25       0.83      21.75       0.83      31.50   

   var_rss23  
0       0.94  
1       0.87  
2       1.50  
   # Columns: time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  \
0                0      47.33       0.47       28.0       0.82      34.25   
1              250      47.50       0.50       29.0       1.22      36.00   
2              500      47.50  

In [29]:
print(len(trainData))
print(len(testData))
print(type(testData)) 
print(type(trainData))
#there are 88 instances in list and df in each  

69
19
<class 'list'>
<class 'list'>


### (c) Feature Extraction

#### i. Research

Minimum: Lowest value
Maximum: Highest Value
Mean: Average value
Median: Middle value
Mode: Most frequent value
Range: Difference between lowest and highest value
Standard Deviation: Variation with avalues 
Q1-Q4: Every 25 percentile of data



#### ii. Extraction

In [30]:
#created function since we are doing it multiple times to all data
def featureExtraction(data):
    features = {
        'min': data.min(),
        'max': data.max(),
        'mean': data.mean(),
        'median': data.median(),
        'std': data.std(),
        'Q1': data.quantile(0.25),
        'Q3': data.quantile(0.75),
    }
    return features

In [31]:
#chatgpt:  I have 2 list, trainData and testData, inside the list are dataframes. 
# Each data frame has 7 columns but i only want the last 6 columns. There are a total of 88 data frames. How do i combine everything
dataTableFeatures = []

# Process each DataFrame in trainData and testData
for df_list in [trainData, testData]:
    for df in df_list:
        # Extract the last 6 columns with the features
        last_six_columns = df.iloc[:, -6:]
        # Initialize a dictionary to hold the aggregated features for the last 6 columns
        aggregated_features = {}
        # Calculate the desired statistics for each of the last 6 columns
        #rename for the corresponding time series
        last_six_columns.columns = [str(i) for i in range(1, 7)]

        for column in last_six_columns.columns:
            aggregated_features[column] = featureExtraction(last_six_columns[column])

        # Append to table
        dataTableFeatures.append(aggregated_features)

# Convert the list of dictionaries to a DataFrame
final_table = pd.DataFrame(dataTableFeatures)
#flatten for visability, object to float :D
final_table_flat = pd.json_normalize(final_table.to_dict(orient='records'))

print(final_table_flat.head(5))

   1.min  1.max     1.mean  1.median     1.std   1.Q1   1.Q3  2.min  2.max  \
0  36.25  48.00  43.968017     44.50  1.619873  43.29  44.67    0.0   1.50   
1  37.00  48.00  43.459040     43.25  1.384657  42.50  45.00    0.0   1.58   
2  33.00  47.75  42.177578     43.50  3.674177  39.10  45.00    0.0   3.00   
3  33.00  45.75  41.678434     41.75  2.245821  41.33  42.75    0.0   2.83   
4  35.00  47.40  43.947620     44.33  1.553152  43.00  45.00    0.0   1.70   

     2.mean  ...     5.std    5.Q1   5.Q3  6.min  6.max    6.mean  6.median  \
0  0.412944  ...  3.321655  20.500  23.75    0.0   2.96  0.555428      0.49   
1  0.377829  ...  2.448733  22.250  24.00    0.0   5.26  0.678518      0.50   
2  0.696597  ...  3.845985  30.500  36.33    0.0   2.18  0.613904      0.50   
3  0.535282  ...  2.413538  28.415  31.25    0.0   1.79  0.384092      0.43   
4  0.425407  ...  2.001635  35.450  36.50    0.0   1.79  0.491608      0.43   

      6.std  6.Q1  6.Q3  
0  0.488330  0.00  0.83  
1  0

#### iii. Standard Deviation

In [37]:
sd = final_table_flat.std()
print("Standard Deviations:\n", sd)

Standard Deviations:
 1.min       9.569975
1.max       4.394362
1.mean      5.336482
1.median    5.439770
1.std       1.772671
1.Q1        6.155771
1.Q3        5.136647
2.min       0.000000
2.max       5.062729
2.mean      1.574291
2.median    1.412978
2.std       0.884215
2.Q1        0.947312
2.Q3        2.126178
3.min       2.957401
3.max       4.866932
3.mean      4.007810
3.median    4.035277
3.std       0.945411
3.Q1        4.220409
3.Q3        4.172617
4.min       0.000000
4.max       2.183625
4.mean      1.166265
4.median    1.146183
4.std       0.458119
4.Q1        0.842470
4.Q3        1.554079
5.min       6.124001
5.max       5.747975
5.mean      5.677393
5.median    5.813650
5.std       1.024759
5.Q1        6.102394
5.Q3        5.530602
6.min       0.045838
6.max       2.518921
6.mean      1.154662
6.median    1.086278
6.std       0.517810
6.Q1        0.758745
6.Q3        1.524198
dtype: float64


In [38]:
#Chatgpt: how to make a bootstrap function with numpy 
def bootstrap_sd(data, n_iterations=1000, ci=0.90):
    stds = []
    
    for _ in range(n_iterations):
        sample = np.random.choice(data, size=len(data), replace=True)
        stds.append(np.std(sample, ddof=1)) 
    
    # confidence intervals
    lower_bound = np.percentile(stds, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(stds, (1 + ci) / 2 * 100)
    
    return np.mean(stds), lower_bound, upper_bound

#store results in 
results = pd.DataFrame(columns=['90% CI Lower', '90% CI Upper'])
# Iterate over each relevant column in final_table_flat
for column in final_table_flat.columns:
        # Drop NaN values before bootstrapping
        data = final_table_flat[column].dropna().values
        mean_sd, lower_ci, upper_ci = bootstrap_sd(data)
        results.loc[column] = {
            '90% CI Lower': lower_ci,
            '90% CI Upper': upper_ci
        }

print(results)

          90% CI Lower  90% CI Upper
1.min         8.310739     10.754129
1.max         3.289622      5.354982
1.mean        4.710695      5.890135
1.median      4.775032      5.995268
1.std         1.582482      1.947710
1.Q1          5.515212      6.657211
1.Q3          4.284033      5.842541
2.min         0.000000      0.000000
2.max         4.627037      5.388936
2.mean        1.402511      1.712691
2.median      1.240491      1.546097
2.std         0.809166      0.942633
2.Q1          0.828208      1.033642
2.Q3          1.894882      2.284588
3.min         2.756017      3.107466
3.max         4.121913      5.429660
3.mean        3.449058      4.488551
3.median      3.440959      4.537314
3.std         0.758923      1.121646
3.Q1          3.654284      4.660052
3.Q3          3.560306      4.735504
4.min         0.000000      0.000000
4.max         1.978987      2.353867
4.mean        1.083419      1.224262
4.median      1.053225      1.201208
4.std         0.420512      0.485166
4

#### iv. Select Features

The feature I select are mean, median, and standard deviation. I want the mean to see the average tendenacy, the median to count out any outliers, and the SD to check for variability. 

## 2. ISLR 3.7.4

### (a) Linear Train

I expected the RSS for the cubic regression model to be lower than the RSS for the Lr model because a cubic model is able to capture higher complexity and in turn, fit the training data better. 

### (b) Linear Test

For testing data, I expect the linear regression model to be lower than  the cubic regression model. Cubic model may be able to capture more complexity, however, it may also capture more noise with a test data set. In this case, a simple model may be better to generalize the dataset. 

### (c) Not Linear Train

I expect the RSS to be lower for the cubic model because it can fit the training data more closely even without a relationship due to its flexibility. 

### (d) Not Linear Testing

There is not enough information to tell because performance of both models well depend on any relationships that the data has and how the models will adapt or generalize to them. A cubic model may be able to fit better if the relationship depending on how non linear it is , however, it may also fit to noise thus having a worse performance. 

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 